In [71]:
import pandas as pd
df= pd.read_csv("/content/drive/MyDrive/fake-news.csv")
df

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [72]:
df.dropna()
# X = df.drop('label',axis=1)
X = df.iloc[:,1:4]
print(X)
Y = df['label']

                                                   title  \
0      House Dem Aide: We Didn’t Even See Comey’s Let...   
1      FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2                      Why the Truth Might Get You Fired   
3      15 Civilians Killed In Single US Airstrike Hav...   
4      Iranian woman jailed for fictional unpublished...   
...                                                  ...   
20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiumnews.com   
3                                Jessica Purkiss   
4                  

In [73]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense


In [74]:
#vocabulary size
vocab_size=5000

In [75]:
##one hot representation
messages=X.copy()
messages['title'][1]
messages.reset_index(inplace=True)

In [76]:
import nltk
import re
from nltk.corpus import stopwords

In [77]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [78]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    # print(i)
    review = re.sub('[^a-zA-Z]',' ',str(messages['title'][i]))
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)
# corpus

In [79]:
onehot_repr=[one_hot(words,vocab_size)for words in corpus]
# onehot_repr

In [80]:
##embedding representation
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ...  881  147 4269]
 [   0    0    0 ... 2820 3312 1298]
 [   0    0    0 ... 4065 1362 2225]
 ...
 [   0    0    0 ... 3281 2150 2794]
 [   0    0    0 ... 3081 3385 2257]
 [   0    0    0 ... 3342 2187 4447]]


In [81]:
##adding dropout
embedding_vector_features=40
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(input_dim=vocab_size,output_dim=embedding_vector_features,embeddings_initializer='uniform',input_length=sent_length,sparse=False))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())


Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 20, 40)            200000    
                                                                 
 dropout_14 (Dropout)        (None, 20, 40)            0         
                                                                 
 lstm_7 (LSTM)               (None, 100)               56400     
                                                                 
 dropout_15 (Dropout)        (None, 100)               0         
                                                                 
 dense_7 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [82]:

len(embedded_docs),Y.shape

(20800, (20800,))

In [83]:
import numpy as np
X_final=np.array(embedded_docs)
Y_final=np.array(Y)
X_final.shape,Y_final.shape

((20800, 20), (20800,))

In [84]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_final, Y_final, test_size=0.33, random_state=42)
# print(X_test,Y_test)

In [85]:
### Finally Training
model.fit(X_train,Y_train,validation_data=(X_test,Y_test),epochs=10,batch_size=64,verbose=1)

Epoch 1/10
218/218 [==============================] - 19s 77ms/step - loss: 0.3440 - accuracy: 0.8422 - val_loss: 0.1848 - val_accuracy: 0.9232
Epoch 2/10
218/218 [==============================] - 5s 21ms/step - loss: 0.1386 - accuracy: 0.9468 - val_loss: 0.1857 - val_accuracy: 0.9222
Epoch 3/10
218/218 [==============================] - 3s 16ms/step - loss: 0.1003 - accuracy: 0.9627 - val_loss: 0.1858 - val_accuracy: 0.9293
Epoch 4/10
218/218 [==============================] - 3s 13ms/step - loss: 0.0710 - accuracy: 0.9760 - val_loss: 0.2048 - val_accuracy: 0.9272
Epoch 5/10
218/218 [==============================] - 3s 13ms/step - loss: 0.0488 - accuracy: 0.9842 - val_loss: 0.2086 - val_accuracy: 0.9276
Epoch 6/10
218/218 [==============================] - 3s 16ms/step - loss: 0.0340 - accuracy: 0.9893 - val_loss: 0.2273 - val_accuracy: 0.9248
Epoch 7/10
218/218 [==============================] - 3s 13ms/step - loss: 0.0275 - accuracy: 0.9905 - val_loss: 0.3442 - val_accuracy: 0.918

In [86]:
##perfomance metrics and accuracy
y_pred=model.predict(X_test)
from sklearn.metrics import confusion_matrix
confusion_matrix(Y_test,y_pred.round())


215/215 [==============================] - 1s 3ms/step


array([[3173,  276],
       [ 248, 3167]])

In [87]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test,y_pred.round())

0.9236596736596736